**1) Implement the program of finding distinct elements in data stream, Flajolet–Martin
algorithm. Consider the data and suitable hash function.**

In [1]:
import random


def hash_function(x, a=17, b=23, p=2**31-1):
    return (a * hash(x) + b) % p

def trailing_zeros(n):
    if n == 0:
        return 32
    return (n & -n).bit_length() - 1

def flajolet_martin(stream):
    max_zeroes = 0

    for item in stream:
        h = hash_function(item)
        tz = trailing_zeros(h)
        max_zeroes = max(max_zeroes, tz)


    return 2 ** max_zeroes


stream = ["a", "b", "c", "a", "d", "e", "b", "f", "g", "h", "i"]

estimate = flajolet_martin(stream)

print("Estimated distinct elements =", estimate)
print("Actual distinct elements =", len(set(stream)))


Estimated distinct elements = 2
Actual distinct elements = 9


**2) Frequent Itemsets — Apriori & FP-Growth**

In [3]:
from itertools import combinations

def apriori(transactions, min_support):
    itemsets = {}
    freq_itemsets = []

    # Step 1: Count support of single items
    for transaction in transactions:
        for item in transaction:
            itemsets[(item,)] = itemsets.get((item,), 0) + 1

    itemsets = {k: v for k, v in itemsets.items() if v >= min_support}
    freq_itemsets.extend(itemsets.items())

    k = 2
    current_itemsets = list(itemsets.keys())

    while current_itemsets:
        candidates = list(combinations(set().union(*current_itemsets), k))

        candidate_count = {}
        for transaction in transactions:
            for candidate in candidates:
                if set(candidate).issubset(transaction):
                    candidate_count[candidate] = candidate_count.get(candidate, 0) + 1

        current_itemsets = [item for item, cnt in candidate_count.items() if cnt >= min_support]
        freq_itemsets.extend([(item, candidate_count[item]) for item in current_itemsets])

        k += 1

    return freq_itemsets


transactions = [
    {'milk', 'bread', 'eggs'},
    {'bread', 'butter'},
    {'milk', 'bread', 'butter'},
    {'bread', 'eggs'},
    {'milk', 'eggs'},
]

min_support = 2
result = apriori(transactions, min_support)

print("Frequent Itemsets:")
for itemset, sup in result:
    print(itemset, "Support =", sup)


Frequent Itemsets:
('bread',) Support = 4
('milk',) Support = 3
('eggs',) Support = 3
('butter',) Support = 2
('bread', 'milk') Support = 2
('bread', 'eggs') Support = 2
('milk', 'eggs') Support = 2
('butter', 'bread') Support = 2


In [4]:
from collections import defaultdict

class FPNode:
    def __init__(self, item, parent):
        self.item = item
        self.count = 1
        self.parent = parent
        self.children = {}

def insert_transaction(root, transaction):
    current = root
    for item in transaction:
        if item not in current.children:
            current.children[item] = FPNode(item, current)
        else:
            current.children[item].count += 1
        current = current.children[item]

def get_patterns(node, prefix):
    if node.item is not None:
        prefix = prefix + [node.item]

    patterns = {tuple(prefix): node.count}

    for child in node.children.values():
        patterns.update(get_patterns(child, prefix))
    return patterns


def fp_growth(transactions):
    freq = defaultdict(int)
    for t in transactions:
        for item in t:
            freq[item] += 1

    sorted_trans = []
    for t in transactions:
        sorted_trans.append(sorted(t, key=lambda x: freq[x], reverse=True))

    root = FPNode(None, None)
    for t in sorted_trans:
        insert_transaction(root, t)

    return get_patterns(root, [])


# Example dataset
transactions = [
    ['milk', 'bread', 'eggs'],
    ['bread', 'butter'],
    ['milk', 'bread', 'butter'],
    ['bread', 'eggs'],
    ['milk', 'eggs'],
]

patterns = fp_growth(transactions)

print("Frequent Patterns from FP-Growth:")
for pattern, count in patterns.items():
    print(pattern, ":", count)


Frequent Patterns from FP-Growth:
() : 1
('bread',) : 4
('bread', 'milk') : 2
('bread', 'milk', 'eggs') : 1
('bread', 'milk', 'butter') : 1
('bread', 'butter') : 1
('bread', 'eggs') : 1
('milk',) : 1
('milk', 'eggs') : 1


**3) Bloom Filter Implementation**

In [5]:
import hashlib

class BloomFilter:
    def __init__(self, size=100, hash_count=3):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = [0] * size

    def _hashes(self, item):
        return [
            int(hashlib.md5(item.encode()).hexdigest(), 16) % self.size,
            int(hashlib.sha1(item.encode()).hexdigest(), 16) % self.size,
            int(hashlib.sha256(item.encode()).hexdigest(), 16) % self.size
        ]

    def add(self, item):
        for pos in self._hashes(item):
            self.bit_array[pos] = 1

    def check(self, item):
        return all(self.bit_array[pos] == 1 for pos in self._hashes(item))


# Example usage
bf = BloomFilter(size=50, hash_count=3)

stream_items = ["apple", "banana", "grape", "apple"]

for it in stream_items:
    bf.add(it)

print("Check 'apple':", bf.check("apple"))
print("Check 'kiwi' :", bf.check("kiwi"))


Check 'apple': True
Check 'kiwi' : False
